# Time series analysis

Celem tej części projektu, jest stworzenie modelu szeregu czasowego dla całkowitej liczby wypożyczeń.

Zadania:
1. Przeprowadź wstępną analizę danych w tym:
    - pobranie danych i podstawowe statystyki
    - reasmpling na dane dzienne
    - dekompozycję
    - zagreguj dane do miesięcznych, dla modelu długookresowego.
2. Stwórz model długookresowy:
    - podziel dane na zbiór treningowy oraz testowy
    - stwórz model miesięcznej predykcji wypożyczeń w kolejnym roku 
    - dokonaj predykcji na danych historycznych oraz 2021 rok (przedstaw graficznie wyniki)
    - oceń jakość za pomocą dwóch, wybranych wskaźników dla modeli regresyjnych.
3. Model krótkookresowy - predykcja na kolejny dzień:
Drugi model będzie potrzebny, aby na koniec danego dnia mieć oszacowanie jak bardzo system będzie oblegany. Można dzięki temu zaplanować serwis stacji, rowerów, czy przemieścić rowery tak, aby najbardziej oblegane stacje miały dostatecznie dużo pojazdów.
    - Dodaj do danych zmienne egzogeniczne (średnie wartości z poprzedniego dnia, temperatura z ostatnich 7 dni)
    - podziel dane na zbiór treningowy oraz testowy
    - Stwórz model dedykowany do szeregów czasowych ze zmiennymi egzogenicznymi.
    - Oceń model.
    - Porównaj model do podejścia machine learning.

## 1. Przeprowadź wstępną analizę danych.

- pobranie danych i podstawowe statystyki

In [1]:
import pandas as pd 
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [3]:
# Pobranie danych
df = pd.read_parquet('data/total_agg.parquet')

In [ ]:
# head
df.head()

In [ ]:
# info
df.info()

- reasmpling na dane dzienne


In [5]:
# ustawienie daty jako indeks
df = df.set_index('departure_date')

In [ ]:
# resampling
df_resampled = df.resample('D').sum().fillna(0)
df_resampled

- dekompozycja

In [7]:
# obiekt dekompozycji
decomp = sm.tsa.seasonal_decompose(df_resampled['numbers_of_renting'],model='additive',period= 365)

In [ ]:
# wykres dekompozycji
decomp.plot()
plt.show()

- zagreguj dane do miesięcznych, dla modelu długookresowego.

In [ ]:
df_monthly = df.resample('ME').sum().fillna(0).reset_index().rename(columns = {'numbers_of_renting': 'y',
                                                                               'departure_date':'ds'})
df_monthly

In [ ]:
# dekompozycja miesięczna
decom_monthly = sm.tsa.seasonal_decompose(df_monthly['y'],period=12)
decom_monthly.plot()
plt.show()

2. Stwórz model długookresowy:
    - podziel dane na zbiór treningowy oraz testowy

In [11]:
# train / test split
train_monthly = df_monthly.loc[df_monthly['ds'].dt.year<2020,['y','ds']]
test_monthly = df_monthly.loc[df_monthly['ds'].dt.year==2020,['y','ds']]

- stwórz model miesięcznej predykcji wypożyczeń w kolejnym roku 

In [ ]:
# Pobranie neuralprophet
from neuralprophet import NeuralProphet
import numpy as np

In [13]:
# obiekt modelu
model_monthly = NeuralProphet()

In [ ]:
# fit
model_monthly.fit(train_monthly, freq = 'ME')

- dokonaj predykcji na danych historycznych oraz 2021 rok (przedstaw graficznie wyniki)

In [ ]:
# forecast
forecast_historical_monthly = model_monthly.predict(df_monthly[['ds','y']])
forecast_historical_monthly

In [ ]:
# Stworzenie zakresu danych dla roku 2021
new_data_index = pd.date_range(start = df_monthly['ds'].max()+ pd.Timedelta(days=30),end = df_monthly['ds'].max()+pd.Timedelta(days=400),freq= 'ME')
new_data_index

In [ ]:
# Stworzenie ramki danych
forecast_df_monthly = pd.DataFrame(data=new_data_index, columns = ['ds'])
forecast_df_monthly['y'] = np.nan
forecast_df_monthly

In [ ]:
# Predykcja przyszłego roku
forecast_future_monthly = model_monthly.predict(forecast_df_monthly)

In [ ]:
forecast_future_monthly

In [20]:
# funkcja do rysowania wykresu
def plot_forecasts(df: pd.DataFrame, 
                   date_col :str= 'ds',
                   actuals_col: str = 'y',
                   pred_col: str='yhat1'):
    plt.figure(figsize=(10,6))
    plt.plot(df[date_col],df[actuals_col],label = 'actual')
    plt.plot(df[date_col],df[pred_col],label = 'prediction')
    plt.legend()
    plt.show()

In [ ]:
# Wykres
plot_forecasts(pd.concat([forecast_historical_monthly,forecast_future_monthly]))

In [22]:
# Dodanie miesiąca
forecast_historical_monthly['month'] = forecast_historical_monthly.ds.dt.month
forecast_future_monthly['month'] = forecast_future_monthly.ds.dt.month

In [23]:
# Wyzerowanie forecastów w miesiącach, w których system nie funkcjonuje
forecast_historical_monthly.loc[forecast_historical_monthly['month'].isin([12,1,2]),'yhat1'] = 0
forecast_future_monthly.loc[forecast_future_monthly['month'].isin([12,1,2]),'yhat1'] = 0

In [ ]:
# wykres
plot_forecasts(pd.concat([forecast_historical_monthly, forecast_future_monthly]))

In [25]:
# dodanie znacznika zbioru train/ test
forecast_historical_monthly['set'] = 'train'
forecast_historical_monthly.loc[forecast_historical_monthly['ds']>='2020-01-01','set'] = 'test'

In [26]:
# Pobranie metryk
from sklearn.metrics import root_mean_squared_error, r2_score

In [ ]:
# r2 score
forecast_historical_monthly.loc[:,['y','yhat1','set']].groupby('set').apply(lambda x: r2_score(x['y'],x['yhat1']))

In [ ]:
# rmse
forecast_historical_monthly.loc[:,['y','yhat1','set']].groupby('set').apply(lambda x: root_mean_squared_error(x['y'],x['yhat1']))

In [ ]:
forecast_historical_monthly.describe()

In [30]:
import os 
import joblib

In [31]:
if not os.path.exists('models'):
    os.mkdir('models')

In [ ]:
joblib.dump(model_monthly, 'models/model_monthly_forecasts.joblib')

## 3. Model krótkookresowy - predykcja na kolejny dzień:
Drugi model będzie potrzebny, aby na koniec danego dnia mieć oszacowanie jak bardzo system będzie oblegany. Można dzięki temu zaplanować serwis stacji, rowerów, czy przemieścić rowery tak, aby najbardziej oblegane stacje miały dostatecznie dużo pojazdów.

- Dodaj do danych zmienne egzogeniczne (średnie wartości z poprzedniego dnia, temperatura z ostatnich 7 dni)

In [33]:
df_resampled = df_resampled.sort_index()

In [ ]:
df_resampled.head()

In [35]:
# dodanie zmienych z wczoraj
df_resampled['temperature_yesterday'] = df_resampled['Air temperature (degC)'].shift(1)
df_resampled['avg_speed_yesterday'] = df_resampled['avg_speed (km/h)'].shift(1)
df_resampled['avg_duration_yesterday'] = df_resampled['duration (sec.)'].shift(1)

In [36]:
# dodanie informacji z daty
df_resampled['month'] = df_resampled.index.month
df_resampled['day'] = df_resampled.index.day
df_resampled['quarter'] = df_resampled.index.quarter

In [ ]:
# usunięcie braków danych
df_resampled = df_resampled.dropna().reset_index()
df_resampled

In [38]:
# dopasowanie nazw w ramce danych
df_resampled = df_resampled.rename(columns= {'departure_date':'ds','numbers_of_renting':'y'})

In [ ]:
df_resampled

- podziel dane na zbiór treningowy oraz testowy


In [40]:
# train / test split
train_daily = df_resampled[df_resampled['ds'].dt.year < 2020]
test_daily = df_resampled[df_resampled['ds'].dt.year==2020]

In [41]:
# zmienne do modelowania
cols_names =  ['ds','y','avg_speed_yesterday','avg_duration_yesterday','Air temperature (degC)']

In [ ]:
train_daily[cols_names]

- Stwórz model dedykowany do szeregów czasowych ze zmiennymi egzogenicznymi.


In [43]:
# model na danych dziennych 
model_daily = NeuralProphet()


In [ ]:
# dodanie regresorów
model_daily.add_future_regressor('avg_speed_yesterday')
model_daily.add_future_regressor('avg_duration_yesterday')
model_daily.add_lagged_regressor('Air temperature (degC)',n_lags=7)
model_daily.add_country_holidays('Finland')

In [ ]:
# fit
model_daily.fit(train_daily[cols_names])

- Oceń model.


In [ ]:
# predykcje historyczne
historical_predictions = model_daily.predict(df_resampled[cols_names])

In [47]:
# dodanie miesiąca oraz identyfikatora train/test
historical_predictions['month'] = historical_predictions.ds.dt.month
historical_predictions['set'] = 'train'
historical_predictions.loc[historical_predictions['ds']>='2020-01-01','set'] = 'test'

In [ ]:
# wykres
plot_forecasts(historical_predictions)

In [49]:
# wyzerowanie miesięcy z zamkniętym systemem
historical_predictions.loc[historical_predictions['month'].isin([12,1,2]),'yhat1'] = 0

In [ ]:
# ponowny wykres
plot_forecasts(historical_predictions)

In [ ]:
historical_predictions[['yhat1']].isna().max()

In [ ]:
historical_predictions[historical_predictions['yhat1'].isna()]

In [53]:
# usunięcie braków danych
historical_predictions  = historical_predictions.dropna()

In [ ]:
historical_predictions.shape

In [ ]:
# r2 score
historical_predictions.loc[:,['y','yhat1','set']].groupby('set').apply(lambda x:r2_score(x['y'],x['yhat1']))

In [ ]:
# rmse
historical_predictions.loc[:,['y','yhat1','set']].groupby('set').apply(lambda x:root_mean_squared_error(x['y'],x['yhat1']))

In [ ]:
historical_predictions.describe()

- Porównaj model do podejścia machine learning.

In [58]:
df_resampled = df.resample('D').sum().fillna(0)

In [ ]:
df_resampled.columns

In [ ]:
# dodanie zmienych z wczoraj
df_resampled = df_resampled.sort_index()
df_resampled['temperature_yesterday'] = df_resampled['Air temperature (degC)'].shift(1)
df_resampled['avg_speed_yesterday'] = df_resampled['avg_speed (km/h)'].shift(1)
df_resampled['avg_duration_yesterday'] = df_resampled['duration (sec.)'].shift(1)
df_resampled = df_resampled.rename(columns = {'departure_date':'ds', 'numbers_of_renting':'y' })
df_resampled['y_yesterday'] = df_resampled['y'].shift(1)
# dodanie informacji z daty
df_resampled['month'] = df_resampled.index.month
df_resampled['day'] = df_resampled.index.day
df_resampled['quarter'] = df_resampled.index.quarter
# usunięcie braków danych
df_resampled = df_resampled.dropna().reset_index()
df_resampled = df_resampled.rename(columns = {'departure_date':'ds'})
df_resampled

In [61]:
# Filtrowanie tylko niepustych wartości
df_resampled_non_zero= df_resampled[df_resampled['y']>0].reset_index(drop= True)

In [62]:
# train/test split
train_daily  =df_resampled_non_zero[df_resampled_non_zero['ds'].dt.year<2020]
test_daily = df_resampled_non_zero[df_resampled_non_zero['ds'].dt.year ==2020]

In [ ]:
train_daily.columns

In [64]:
# zmienne x
x_names = ['temperature_yesterday',
       'avg_speed_yesterday', 'avg_duration_yesterday','month', 'day',
       'quarter', 'y_yesterday']

In [65]:
# train_x / train_y
train_x = train_daily[x_names]
train_y = train_daily['y']
test_x  = test_daily[x_names]
test_y = test_daily['y']

In [66]:
# pobranie funkcji
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import differential_evolution

In [67]:
# Granice przeszukiwan
bounds = [(50,200),
          (5,50),
          (5,50)]

In [68]:
# Funkcja optymalizacyjna
def optimization_function(params, train_x, train_y, test_x,test_y):
    params = {'n_estimators': int(round(params[0])),
    
              'max_depth': int(round(params[1])),
              'min_samples_split': int(round(params[2]))
              }
    model = RandomForestRegressor(**params).fit(train_x,train_y)
    preds = model.predict(test_x)
    return -r2_score(test_y,preds)

In [69]:
# Optymalizacja ewolucyjna
optimization = differential_evolution(func = optimization_function,
                                      bounds = bounds,
                                      args = (train_x, train_y,test_x,test_y),
                                      maxiter=10)

In [74]:
# Najlepsze parametry
params = optimization.x
best_params ={'n_estimators': int(round(params[0])),
              'max_depth': int(round(params[1])),
              'min_samples_split': int(round(params[2]))}

In [ ]:
best_params

In [76]:
# finalny model
model_final = RandomForestRegressor(**best_params).fit(train_x,train_y)

In [77]:
# predykcje
df_resampled_non_zero['pred_RF'] = model_final.predict(df_resampled_non_zero[x_names])

In [ ]:
# wykres 
plt.figure(figsize=(10,6))
plt.plot(historical_predictions['ds'],historical_predictions['y'],label=  'actual')
plt.plot(df_resampled_non_zero['ds'], df_resampled_non_zero['pred_RF'],label = 'prediction_RF')
plt.legend()
plt.show()

In [ ]:
# wykres
plt.figure(figsize=(10,6))
plt.plot(historical_predictions['ds'],historical_predictions['y'],label=  'actual')
plt.plot(df_resampled_non_zero['ds'], df_resampled_non_zero['pred_RF'],label = 'prediction_RF')
plt.plot(historical_predictions['ds'],historical_predictions['yhat1'],label='prediction_NP', alpha=0.6)
plt.legend()
plt.show()

In [83]:
# znacznik train/test
df_resampled_non_zero['set'] = 'train'
df_resampled_non_zero.loc[df_resampled_non_zero['ds']>='2020-01-01','set'] = 'test'

In [ ]:
# r2_score
df_resampled_non_zero.loc[:,['y','pred_RF','set']].groupby('set').apply(lambda x: r2_score(x['y'],x['pred_RF']))

In [ ]:
# r2 score na neural prophet
historical_predictions.loc[historical_predictions['y']>0,['y','yhat1','set']].groupby('set').apply(lambda x: r2_score(x['y'],x['yhat1']))

In [86]:
# biblioteki do zapisania modeli
import joblib
import os 

In [87]:
# stworzenie ścieżki
if not os.path.exists('models'):
    os.mkdir('models')

In [ ]:
# zapisanie modelu krótkookresowego
joblib.dump(model_final, 'models/model_RF_daily_forecasts.joblib')


In [ ]:
# zapisanie modelu długookresowego
